In [1]:
!pip install keras torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 66.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 24.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 19.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 16.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip install keras_sig pandas

In [3]:
import os
# Keras and backend configuration
BACKEND = 'torch'
os.environ['KERAS_BACKEND'] = BACKEND

print('removing access to CUDA device')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["JAX_PLATFORMS"]="cpu" 

removing access to CUDA device


In [4]:
import numpy as np
import pandas as pd
import time
import json
from datetime import datetime
import platform
from typing import Callable, List, Dict
import keras_sig
import torch
from IPython.display import display

def get_cpu_info():
    if platform.system() == "Linux":
        try:
            with open('/proc/cpuinfo', 'r') as f:
                for line in f:
                    if 'model name' in line:
                        return line.split(':')[1].strip()
        except:
            pass
    return platform.processor() or platform.machine()

def time_function(func: Callable, number: int = 10) -> float:
    """Time a function over multiple runs and return average time in milliseconds"""
    # First call to compile
    compilation_start = time.time()
    _ = func()
    compilation_time = (time.time() - compilation_start) * 1000
    
    # Subsequent calls for execution time
    times = []
    for _ in range(number):
        start = time.time()
        _ = func()
        times.append((time.time() - start) * 1000)  # Convert to milliseconds
    
    return compilation_time, np.mean(times)

def run_benchmark(batch_size: int, seq_len: int, n_features: int, depth: int) -> Dict:
    """Run benchmark for a specific configuration"""
    
    # Generate paths
    paths = np.random.randn(batch_size, seq_len, n_features).astype(np.float32)
    paths_torch = torch.from_numpy(paths)
    
    if torch.cuda.is_available():
        paths_torch = paths_torch.cuda()
    
    # Time implementation with explicit compilation and execution times
    compilation_time, execution_time = time_function(
        lambda: keras_sig.signature(paths_torch, depth)
    )
    
    results = {
        'batch_size': batch_size,
        'seq_len': seq_len,
        'n_features': n_features,
        'depth': depth,
        'compilation_time': compilation_time,
        'execution_time': execution_time
    }
    
    return results

def run_parameter_sweep():
    """Run benchmarks varying one parameter at a time"""
    
    # Default parameters
    default_batch_size = 128
    default_seq_len = 100
    default_n_features = 3
    default_depth = 4
    
    # Parameter ranges
    batch_sizes = [32, 64, 128, 256, 512]
    seq_lens = [50, 100, 200, 500, 1000]
    depths = [2, 3, 4, 5, 6]
    
    results = []
    
    # Display PyTorch configuration
    is_gpu = torch.cuda.is_available()
    print("PyTorch CUDA Available:", is_gpu)
    if is_gpu:
        print("GPU Device:", torch.cuda.get_device_name(0))
    
    # Vary batch size
    print("\nVarying batch size...")
    for batch_size in batch_sizes:
        result = run_benchmark(
            batch_size=batch_size,
            seq_len=default_seq_len,
            n_features=default_n_features,
            depth=default_depth
        )
        results.append(result)
        print(f"Completed batch_size={batch_size}")
    
    # Vary sequence length
    print("\nVarying sequence length...")
    for seq_len in seq_lens:
        result = run_benchmark(
            batch_size=default_batch_size,
            seq_len=seq_len,
            n_features=default_n_features,
            depth=default_depth
        )
        results.append(result)
        print(f"Completed seq_len={seq_len}")
    
    # Vary depth
    print("\nVarying depth...")
    for depth in depths:
        result = run_benchmark(
            batch_size=default_batch_size,
            seq_len=default_seq_len,
            n_features=default_n_features,
            depth=depth
        )
        results.append(result)
        print(f"Completed depth={depth}")
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    device_type = "gpu" if is_gpu else "cpu"
    csv_filename = f'pytorch_signature_benchmarks_{device_type}_{timestamp}.csv'
    df.to_csv(csv_filename, index=False)
    
    # Save metadata about the system
    metadata = {
        'cpu_info': get_cpu_info(),
        'pytorch_version': torch.__version__,
        'keras_sig_version': '1.0.2',
        'cuda_available': is_gpu,
        'gpu_device': torch.cuda.get_device_name(0) if is_gpu else None,
        'cuda_version': torch.version.cuda if is_gpu else None,
        'timestamp': timestamp,
        'system': platform.system(),
        'python_version': platform.python_version(),
        'platform': platform.platform()
    }
    
    with open(f'pytorch_signature_benchmarks_{device_type}_metadata_{timestamp}.json', 'w') as f:
        json.dump(metadata, f, indent=4)
    
    return df, metadata

print("Starting PyTorch signature benchmarks...")
df, metadata = run_parameter_sweep()
print("\nBenchmarks complete. Results saved to CSV.")
print("\nSystem information:")
for key, value in metadata.items():
    print(f"{key}: {value}")

display(df)

Starting PyTorch signature benchmarks...
PyTorch CUDA Available: False

Varying batch size...
Completed batch_size=32
Completed batch_size=64
Completed batch_size=128
Completed batch_size=256
Completed batch_size=512

Varying sequence length...
Completed seq_len=50
Completed seq_len=100
Completed seq_len=200
Completed seq_len=500
Completed seq_len=1000

Varying depth...
Completed depth=2
Completed depth=3
Completed depth=4
Completed depth=5
Completed depth=6

Benchmarks complete. Results saved to CSV.

System information:
cpu_info: AMD Ryzen 9 5900X 12-Core Processor
pytorch_version: 2.5.1+cu124
keras_sig_version: 1.0.2
cuda_available: False
gpu_device: None
cuda_version: None
timestamp: 20250105_163733
system: Linux
python_version: 3.10.12
platform: Linux-6.8.0-45-generic-x86_64-with-glibc2.35


,batch_size,seq_len,n_features,depth,compilation_time,execution_time
0,32,100,3,4,3.991604,1.471663
1,64,100,3,4,2.605677,2.927780
2,128,100,3,4,4.282713,3.965688
3,256,100,3,4,7.016659,6.830716
4,512,100,3,4,12.342930,12.337351
5,128,50,3,4,2.187014,1.624846
6,128,100,3,4,2.614975,2.113104
7,128,200,3,4,3.787279,4.548478
8,128,500,3,4,13.433933,14.608765
9,128,1000,3,4,31.124592,32.256675


In [6]:
import keras
import numpy as np
import pandas as pd
import time
import json
from datetime import datetime
import platform

def get_cpu_info():
    if platform.system() == "Linux":
        try:
            with open('/proc/cpuinfo', 'r') as f:
                for line in f:
                    if 'model name' in line:
                        return line.split(':')[1].strip()
        except:
            pass
    return platform.processor() or platform.machine()

class SigNet(keras.Model):
    def __init__(self, in_channels, out_dimension, sig_input_size, sig_depth, sig_layer_class):
        super().__init__()
        self.dense1 = keras.layers.Dense(sig_input_size)
        self.signature = sig_layer_class(depth=sig_depth)
        self.linear = keras.layers.Dense(out_dimension)
        
    def call(self, inputs):
        dense_out = self.dense1(inputs)
        y = self.signature(dense_out)
        z = self.linear(y)
        return z

def create_data(num_sample, seq_len, n_feature, n_ahead):
    X = np.random.randn(num_sample, seq_len, n_feature).astype(np.float32)
    y = np.random.randn(num_sample, n_ahead).astype(np.float32)
    return X, y

def measure_compilation_time(model, X, y, batch_size):
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss="mse",
        jit_compile=False
    )
    
    # Time the first prediction which triggers compilation
    sample_X = X[:1]  # Take just one sample
    
    compilation_start = time.time()
    model.predict(sample_X, verbose=0)  # First prediction triggers compilation
    compilation_time = time.time() - compilation_start
    
    return compilation_time

def train_model(model, X, y, batch_size, epochs=10):
    # Time the actual training
    training_start = time.time()
    history = model.fit(
        X, y,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0
    )
    training_time = time.time() - training_start
    
    return {
        'final_loss': float(history.history['loss'][-1]),
        'training_time': training_time,
        'avg_epoch_time': training_time/epochs
    }

def run_benchmarks():
    # Parameters to test
    seq_lens = [100, 200, 350, 500]
    sig_input_sizes = [2, 4, 6, 10]
    depths = [2, 3, 4]
    
    # Fixed parameters
    batch_size = 128
    n_feature = 20
    n_ahead = 10
    epochs = 10
    
    results = []
    sig_layers = {
        'keras_sig': keras_sig.SigLayer,
    }
    
    total_runs = len(seq_lens) * len(sig_input_sizes) * len(depths) * len(sig_layers)
    current_run = 0
    
    for seq_len in seq_lens:
        num_sample = batch_size * 100 - 35  # Not exactly divisible by batch size
        
        for sig_input_size in sig_input_sizes:
            for depth in depths:
                for layer_name, layer_class in sig_layers.items():
                    current_run += 1
                    print(f"\nRun {current_run}/{total_runs}")
                    print(f"Parameters: seq_len={seq_len}, sig_input_size={sig_input_size}, depth={depth}, implementation={layer_name}")
                    
                    # Create data
                    X, y = create_data(num_sample, seq_len, n_feature, n_ahead)
                    
                    # Create model
                    model = SigNet(n_feature, n_ahead, sig_input_size, depth, layer_class)
                    
                    # Measure compilation time
                    compilation_time = measure_compilation_time(model, X, y, batch_size)
                    
                    # Train model and measure training time
                    training_results = train_model(model, X, y, batch_size, epochs)
                    
                    results.append({
                        'seq_len': seq_len,
                        'sig_input_size': sig_input_size,
                        'depth': depth,
                        'implementation': layer_name,
                        'compilation_time': compilation_time,
                        'training_time': training_results['training_time'],
                        'avg_epoch_time': training_results['avg_epoch_time'],
                        'final_loss': training_results['final_loss']
                    })
                    # Clear model and free memory
                    del model
                    keras.backend.clear_session()
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    device_type = "gpu" if keras.backend.backend() == "jax" else "cpu"
    
    # Save benchmark results
    csv_filename = f'keras_benchmarks_{device_type}_{timestamp}.csv'
    df.to_csv(csv_filename, index=False)
    
    # Save metadata
    metadata = {
        'cpu_info': get_cpu_info(),
        'keras_backend': keras.backend.backend(),
        'keras_version': keras.__version__,
        'keras_sig_version': '1.0.2',
        'timestamp': timestamp,
        'system': platform.system(),
        'python_version': platform.python_version(),
        'platform': platform.platform(),
        'batch_size': batch_size,
        'n_feature': n_feature,
        'n_ahead': n_ahead,
        'epochs': epochs
    }
    
    with open(f'keras_torch_benchmarks_{device_type}_metadata_{timestamp}.json', 'w') as f:
        json.dump(metadata, f, indent=4)
    
    return df, metadata

print("Starting Keras signature benchmarks...")
df, metadata = run_benchmarks()
print("\nBenchmarks complete. Results saved to CSV and JSON files.")
print("\nSystem information:")
for key, value in metadata.items():
    print(f"{key}: {value}")

# Display results
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print("\nBenchmark Results:")
print(df)

Starting Keras signature benchmarks...

Run 1/48
Parameters: seq_len=100, sig_input_size=2, depth=2, implementation=keras_sig

Run 2/48
Parameters: seq_len=100, sig_input_size=2, depth=3, implementation=keras_sig

Run 3/48
Parameters: seq_len=100, sig_input_size=2, depth=4, implementation=keras_sig

Run 4/48
Parameters: seq_len=100, sig_input_size=4, depth=2, implementation=keras_sig

Run 5/48
Parameters: seq_len=100, sig_input_size=4, depth=3, implementation=keras_sig

Run 6/48
Parameters: seq_len=100, sig_input_size=4, depth=4, implementation=keras_sig

Run 7/48
Parameters: seq_len=100, sig_input_size=6, depth=2, implementation=keras_sig

Run 8/48
Parameters: seq_len=100, sig_input_size=6, depth=3, implementation=keras_sig

Run 9/48
Parameters: seq_len=100, sig_input_size=6, depth=4, implementation=keras_sig

Run 10/48
Parameters: seq_len=100, sig_input_size=10, depth=2, implementation=keras_sig

Run 11/48
Parameters: seq_len=100, sig_input_size=10, depth=3, implementation=keras_sig
